# Install

In [1]:
!pip install openai pinecone-client python-docx

  Obtaining dependency information for pinecone-client from https://files.pythonhosted.org/packages/98/17/3675b83dca0a032d2750bf04fbfdf78a6e46fa3056eefc2574cdd14661d9/pinecone_client-2.2.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB 1.3 MB/s eta 0:00:05
     ---------------------------------------- 0.0/5.6 MB 1.3 MB/s eta 0:00:05
     ---------------------------------------- 0.0/5.6 MB 1.3 MB/s eta 0:00:05
     ---------------------------------------- 0.1/5.6 MB 297.7 kB/s eta 0:00:19
      --------------------------------------- 0.1/5.6 MB 374.1 kB/s eta 0:00:15
      --------------------------------------- 0.1/5.6 MB 385.0 kB/s eta 0:00:15
      --------------------------------------- 0.1/5.6 MB 385.0 kB/s eta 0:00:15
      --------------------------------------- 0.1/5.6 MB 385.0 kB/s eta 0:00:15
     - -------------------------------------- 0.2/5.6 MB 388.2 kB/s eta 0:00:14


ERROR: Exception:
Traceback (most recent call last):
  File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "d:\Trainings\AI\labs\openai\Basic_Samples\.venv\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
           ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1520.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 466, in read
    s = self.fp.read


     -------------------------------------- - 5.3/5.6 MB 557.7 kB/s eta 0:00:01
     -------------------------------------- - 5.3/5.6 MB 557.7 kB/s eta 0:00:01
     -------------------------------------- - 5.4/5.6 MB 556.7 kB/s eta 0:00:01
     -------------------------------------- - 5.4/5.6 MB 558.1 kB/s eta 0:00:01
     ---------------------------------------  5.4/5.6 MB 556.8 kB/s eta 0:00:01
     ---------------------------------------  5.5/5.6 MB 558.1 kB/s eta 0:00:01
     ---------------------------------------  5.5/5.6 MB 558.4 kB/s eta 0:00:01
     ---------------------------------------  5.6/5.6 MB 560.1 kB/s eta 0:00:01
     ---------------------------------------  5.6/5.6 MB 559.4 kB/s eta 0:00:01
     ---------------------------------------- 5.6/5.6 MB 553.4 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished wi

# Import

In [ ]:
import pinecone
from openai.embeddings_utils import get_embedding
from tqdm import tqdm
import docx
import os
import openai

# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Mount drive a specify the folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
docs_path = "/content/drive/MyDrive/Your Docs Folder"

# Parse Documents

In [ ]:
text_chunks = []
for f_name in os.listdir(docs_path):
  doc_path = os.path.join(docs_path, f_name)
  doc = docx.Document(doc_path)
  for para in doc.paragraphs:
    text_chunks.append(para.text)

In [ ]:
# remove all chunks shorter than 10 words and strip the rest
text_chunks = [string.strip().strip('\n') for string in text_chunks if len(string.split()) >= 10]

# Generate embeddigns

In [ ]:
chunks_with_embeddigns = []
for chunk in tqdm(text_chunks):
  embedding = get_embedding(chunk, engine='text-embedding-ada-002')
  chunks_with_embeddigns.append({"text": chunk, "embedding": embedding})

# Upload to Pinecone

In [ ]:
pinecone.init(
    api_key="",
    environment=""
)

In [ ]:
# create or connect to index
index_name = "tiktok-trends-2023"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=1536)
# connect to index
index = pinecone.Index(index_name)

In [ ]:
batch_size = 64  # process everything in batches of 64
for i in tqdm(range(0, len(chunks_with_embeddigns), batch_size)):
    data_batch = chunks_with_embeddigns.iloc[i: i+batch_size]
    # set end position of batch
    i_end = min(i+batch_size, len(chunks_with_embeddigns))
    # get batch meta
    text_batch = [item['text'] for item in data_batch]
    # get ids
    ids_batch = [str(n) for n in range(i, i_end)]
    # get embeddings
    embeds = [item['embedding'] for item in data_batch]
    # prep metadata and upsert batch
    meta = [{'text': text_batch} for text_batch in zip(text_batch)] # you can add more fields here
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))

# Query Index

In [ ]:
def search_docs(query):
  xq = openai.Embedding.create(input=query, engine="text-embedding-ada-002")['data'][0]['embedding']
  res = index.query([xq], top_k=5, include_metadata=True)
  chosen_text = []
  for match in res['matches']:
    chosen_text = match['metadata']
  return res['matches']

In [ ]:
matches = search_docs("What are some predictions for tiktok?")
for match in matches:
    print(f"{match['score']:.2f}: {match['metadata']}")

# Construct Prompt

In [ ]:
def construct_prompt(query):
  matches = search_docs(query)

  chosen_text = []
  for match in matches:
    chosen_text.append(match['metadata']['text'])

  prompt = """Answer the question as truthfully as possible using the context below, and if the answer is no within the context, say 'I don't know.'"""
  prompt += "\n\n"
  prompt += "Context: " + "\n".join(chosen_text)
  prompt += "\n\n"
  prompt += "Question: " + query
  prompt += "\n"
  prompt += "Answer: "
  return prompt

# Run the model

In [ ]:
def answer_question(query):
  prompt = construct_prompt(query)
  res = openai.Completion.create(
      prompt=prompt,
      model="text-davinci-003",
      max_tokens=500,
      temperature=0.0,
  )

  return res.choices[0].message

In [ ]:
print(answer_question("What will be the top platform in 2023?"))